In [1]:
import datetime
import os, math

import persistence_diagram
import tensorflow as tf
import layers
import numpy as np
import utils

man_dim = 2 # Dimension of the manifold
K = 20 # number of projection bases
num_of_hom = 3 # number of homology classes; hardcoded for now (we know its 3 for images);
               # TODO code function to find it

batch_size = 64
epochs = 3
save_every = 1

def pad_diagrams(dgms):
    '''
        Adds zeros points to the diagrams to make them of equal size; tensorflow cant handle inputs with varied size
    '''
    # Get the highest number of points in a PD
    max_num_of_pnts = 0
    for ind in dgms.keys():
        num_of_dgms = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                max_num_of_pnts = max(max_num_of_pnts, dgm.shape[0])
                num_of_dgms += 1

    # Pad
    N = len(dgms.keys())
    out = np.zeros([N,num_of_dgms,max_num_of_pnts,man_dim+1], dtype=np.float32)
    for ind in dgms.keys():
        cnt = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                out[ind,cnt,:dgm.shape[0],:] = dgm
                cnt += 1
    return out


# Obtain the data
img_id = 'mnist'
train_images, train_labels, test_images, test_labels = utils.get_mnist_data()
# train_images = train_images[:3000]
# train_labels = train_labels[:3000]
# test_images = test_images[:500]
# test_labels = test_labels[:500]

## Set the params of the filtrations to extract
# Height filtration
num_of_vects = 6
angles = np.linspace(0, math.pi/2, num_of_vects)
dirs = [[round(math.cos(theta),2),round(math.sin(theta),2)] for theta in angles]
dirs = np.array(dirs)

# Radial filtration
center = np.array([[10,10], [10,20], [20,10], [20,20]])
radius = np.array([5, 10, 15])

# Erosion filtration
n_iter_er = np.array([1,2,3,50])

# Dilation filtration
n_iter_dil = np.array([1,3,5,10,50])

params = {'cubical' : None,
         'height': dirs,
         'radial': {'center' : center,
                    'radius' : radius
                    },
         'erosion': n_iter_er,
         'dilation': n_iter_dil
         }
# Get persistence diagrams
pd_train = persistence_diagram.PDiagram(train_images, fil_parms=params, images_id='mnist_train')
pd_test = persistence_diagram.PDiagram(test_images, fil_parms=params, images_id='mnist_test')

# Get train test data
dgms_train = pd_train.get_pds()
dgms_test = pd_test.get_pds()
x_train = pad_diagrams(dgms_train)
x_test = pad_diagrams(dgms_test)
y_train = train_labels
y_test = test_labels
print(dgms_train.shape)
print(dgms_test.shape)

AttributeError: 'dict' object has no attribute 'shape'

In [2]:
import layers

#pm = layers.PManifoldLayer(28,2,2)
input = x_train[:2,:,:,:]

In [2]:
num_of_fil=28
theta_init = tf.random_normal_initializer()
theta = tf.Variable(name='theta',
                         initial_value=theta_init(shape=(num_of_fil, man_dim),
                                                  dtype=tf.float32),
                         trainable=True)
class_w = tf.Variable(name='class_weight',
                           initial_value=theta_init(shape=(num_of_hom, ),
                                                    dtype=tf.float32),
                           trainable=True)

x_o = tf.zeros(shape=(man_dim,))
output_batch = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
for i in tf.range(tf.shape(input)[0]):
    out = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    sum = tf.zeros(shape=(man_dim,), dtype=tf.float32)
    for k in tf.range(num_of_fil):
        dgm = input[i,k,:,:]
        tf.print(dgm)
        j = tf.constant(0)
        while j < tf.shape(dgm)[0] and tf.math.count_nonzero(dgm[j,:]) != 0:
            point = dgm[j,:]
            hom = tf.cast(point[0], tf.int32) # last element is the hom. class
            point = point[:2]
            y_pnt = tf.convert_to_tensor(point, dtype=tf.float32)

            ## TODO x_pnt has CONSTRAINTS, need to implement this
            x_pnt = utils.Poincare.tf_parametrization(y_pnt, theta[k,:])
            t_vec_x = utils.Poincare.tf_log_map_x(x_o, x_pnt, 1.)
            if not tf.math.is_nan(tf.reduce_sum(t_vec_x,axis=0)):
                sum = tf.add(sum,
                         tf.scalar_mul(class_w[hom], t_vec_x))
            j = tf.add(j,1)
        x_dgm = utils.Poincare.tf_exp_map_x(x_o, sum, 1.)
        y_dgm = utils.Poincare.tf_chart(x_dgm)
        #print(y_dgm)
        out = out.write(k, y_dgm)
    out = out.stack()
    tf.print(out)
    out = tf.reshape(out, shape=[-1])
    output_batch = output_batch.write(i, out)
    #tf.print(i)
output_batch = output_batch.stack()
outp = tf.reshape(output_batch, shape=[-1, man_dim*num_of_fil])

NameError: name 'tf' is not defined

In [ ]:
aa = tf.reduce_sum(t_vec_x,axis=0)
if not tf.math.is_nan(aa):
    print('fg')




In [ ]:

dgm = input[i,k,:,:]
tf.print(dgm)
j = tf.constant(0)
while j < tf.shape(dgm)[0] and tf.math.count_nonzero(dgm[j,:]) != 0:
    point = dgm[j,:]
    hom = tf.cast(point[0], tf.int32) # last element is the hom. class
    point = point[:2]
    y_pnt = tf.convert_to_tensor(point, dtype=tf.float32)

    ## TODO x_pnt has CONSTRAINTS, need to implement this
    x_pnt = utils.Poincare.tf_parametrization(y_pnt, theta[k,:])
    t_vec_x = utils.Poincare.tf_log_map_x(x_o, x_pnt, 1.)
    aa = tf.reduce_sum(t_vec_x,axis=0)
    if tf.math.is_nan(aa):
        print('fgnfghnfghfg')
        exit()
    sum = tf.add(sum,
                 tf.scalar_mul(class_w[hom], t_vec_x))
    j = tf.add(j,1)
x_dgm = utils.Poincare.tf_exp_map_x(x_o, sum, 1.)
y_dgm = utils.Poincare.tf_chart(x_dgm)
#print(y_dgm)
out = out.write(k, y_dgm)